# Logistic Reg Ensemble

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
dataset = FashionMNIST(root="./data/", train=True,transform=ToTensor(),download=False)
train_set, val_set = train_test_split(dataset, test_size=0.1, random_state=777)

train_loader = DataLoader(train_set,batch_size=100,shuffle=True)
test_loader = DataLoader(val_set, batch_size=100, shuffle=False)

# for i, e in train_loader:
#     print(i, e)

In [5]:
class LogisticReg(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticReg, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        out = self.linear(x)
        
        return out

In [6]:
input_size = 28*28
num_classes = 10
num_epoch = 10
lr = 0.001
num_models = 5

In [7]:
models = [LogisticReg(input_size, num_classes) for _ in range(num_models)]

print(models)

[LogisticReg(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticReg(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticReg(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticReg(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticReg(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)]


In [8]:
criterion = nn.CrossEntropyLoss()
optimizers = [optim.SGD(model.parameters(), lr=lr) for model in models]
print(optimizers)

[SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
), SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
), SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
), SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
), SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)]


In [14]:
num_epoch = 100
for epoch in range(num_epoch):
    for i, (imgs, labels) in enumerate(train_loader):
        #data load
        imgs = imgs.reshape(-1, input_size)
        labels = labels
        
        #forward prop, loss
        for j in range(num_models):
            outputs = models[j](imgs)
            loss = criterion(outputs, labels)
            
            optimizers[j].zero_grad()
            loss.backward()
            optimizers[j].step()
    #eval code
    with torch.no_grad() :
        total, correct = 0,0
        for imgs, labels in test_loader:
            imgs = imgs.reshape(-1, input_size)
            outputs = torch.zeros(imgs.size()[0], num_classes)
            
            #ensemble model prediction val
            for j in range(num_models):
                outputs += models[j](imgs)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        val_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epoch}], Val ACC >> {val_acc:.2f}")
                

Epoch [1/100], Val ACC >> 80.42
Epoch [2/100], Val ACC >> 80.42
Epoch [3/100], Val ACC >> 80.48
Epoch [4/100], Val ACC >> 80.45
Epoch [5/100], Val ACC >> 80.78
Epoch [6/100], Val ACC >> 80.72
Epoch [7/100], Val ACC >> 80.80
Epoch [8/100], Val ACC >> 80.87
Epoch [9/100], Val ACC >> 81.05
Epoch [10/100], Val ACC >> 81.02
Epoch [11/100], Val ACC >> 81.08
Epoch [12/100], Val ACC >> 81.32
Epoch [13/100], Val ACC >> 81.32
Epoch [14/100], Val ACC >> 81.37
Epoch [15/100], Val ACC >> 81.40
Epoch [16/100], Val ACC >> 81.62
Epoch [17/100], Val ACC >> 81.65
Epoch [18/100], Val ACC >> 81.75
Epoch [19/100], Val ACC >> 81.90
Epoch [20/100], Val ACC >> 81.97
Epoch [21/100], Val ACC >> 81.90
Epoch [22/100], Val ACC >> 82.22
Epoch [23/100], Val ACC >> 82.13
Epoch [24/100], Val ACC >> 82.17
Epoch [25/100], Val ACC >> 82.20
Epoch [26/100], Val ACC >> 82.20
Epoch [27/100], Val ACC >> 82.28
Epoch [28/100], Val ACC >> 82.45
Epoch [29/100], Val ACC >> 82.45
Epoch [30/100], Val ACC >> 82.47
Epoch [31/100], Val